In [1]:
import sys
!{sys.executable} -m pip install PyPDF2
!{sys.executable} -m pip install Pandas
import PyPDF2
import pandas as pd
import re
import glob

You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 18.0 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [60]:
data = {}
df = pd.DataFrame()

def extract_info(file):
    main_pdf = PyPDF2.PdfFileReader(file, 'rb')
    main_str = str()

    # decrypt pdf with "" as password
    if main_pdf.isEncrypted:
        main_pdf.decrypt("")

    for pages in range(main_pdf.getNumPages()):
        p = main_pdf.getPage(pages)
        p_text= p.extractText()
        p_lines=str(p_text.splitlines())
        
        if pages == 0:
            main_str += p_lines
        else:
            main_str += " \n "
            main_str += p_lines
    
    # Create dictionary to convert to dataframe later
    data = {
        # Change names if ASX changes their format
        'File Name' : file,
        'Entity Name' : (main_str.split("Entity name")[-1].split("Security on which the Distribution")[0]),
        'Ticker' : (main_str.split("Security on which the Distribution will be paid")[-1])[:3],
        'Record Date' : (main_str.split("2A.4 +Record Date")[-1].split("2A.5 Ex Date")[0]),
        'Ex Date' : (main_str.split("2A.5 Ex Date")[-1].split("2A.6 Payment Date")[0]),
        'Payment Date' : (main_str.split("2A.6 Payment Date")[-1].split("2A.7 Are any of the")[0]),
        'Ex Date' : (main_str.split("2A.5 Ex Date")[-1].split("2A.6 Payment Date")[0])
        }

        
    main_str = main_str.replace(" ", "")
    main_str = main_str.replace("+", "")
    main_str = main_str.replace("Notificationofdividend/distributionNotificationofdividend/distribution", "")
    
    # Add Primary Currency
    if ((main_str.split("Currencyinwhichthedividend/distributionismade")[-1][19:22])[0]) == ")":
        data["Primary Currency"] = (main_str.split("Currencyinwhichthedividend/distributionismade")[-1][20:23])
    else:
        data["Primary Currency"] = ((main_str.split("Currencyinwhichthedividend/distributionismade")[-1][19:22]))
        
    # Add Dividend per Share and sanity check
    data["Dividend per Share"] = ((main_str.split("notifiedinthisform")[-1][3:].split(".")[0]) + "." +
          (main_str.split("notifiedinthisform")[-1][4:].split(".")[1][0:8]))
    
    try:
        temp = float(data["Dividend per Share"])
    except:
        data.update({"Dividend per Share" : "ERROR CANT FIND DIVIDENDS"})
        
    # update with whether dividends per share is actual or estimated
    if (main_str.split("estimatedatthistime?")[-1][:1]) == "Y":
        data.update({"Dividend per Share Status" : "Estimated"})
    else:
        data.update({"Dividend per Share Status" : "Actual"})
    
    # Add Franked Amount
    try:
        franky = (float(main_str.split("distributionthatisfranked")[1].split("%")[0]))
        data.update({"Franked Amount" : (float(data["Dividend per Share"]) * franky)/100})
    except:
        data.update({"Franked Amount" : "ERROR CANT FIND FRANKED AMOUNT"})

    # Add Unfranked Amount
    try:
        unfranky = 100 - franky
        data.update({"Unfranked Amount" : (float(data["Dividend per Share"]) * unfranky)/100})
    except:
        data.update({"Franked Amount" : "ERROR CANT FIND UNFRANKED AMOUNT"})
    
    # Unfranked Amt excluding conduit foreign income amt
    try:
        data.update({"Unfranked Excl Conduit" : 
                     (main_str.split("excludingconduitforeignincomeamount")[1][3:].split(".")[0] + 
                      "." + main_str.split("dividend/excludingconduitforeignincomeamount")[1][3:].split(".")[1][:8])})
    except:
        data.update({"Unfranked Excl Conduit" : 0})
        
    # conduit foreign income amt
    try:
        data.update({"Conduit Foreign Income Amt" : 
                 (main_str.split("distributionconduitforeignincomeamountpersecurity")[1][3:].split(".")[0]) + "." + 
                 (main_str.split("distributionconduitforeignincomeamountpersecurity")[1][3:].split(".")[1][:8])})
    except:
        data.update({"Conduit Foreign Income Amt" : 0})
        
    # Sanity check excl conduit + conduit
    try:
        float(data["Unfranked Excl Conduit"]) + float(data["Conduit Foreign Income Amt"]) == float(data["Unfranked Amount"])
    except:
        data.update({"Unfranked Excl Conduit" : "ERROR SANITY CHECK FAILED"})
        data.update({"Conduit Foreign Income Amt" : "ERROR SANITY CHECK FAILED"})
    
    # Sanity check on dividends (Check if dividends/share = franked + unfranked)
    try:      
        (float(data["Franked Amount"]) + float(data["Unfranked Amount"])) == float(data["Dividend per Share"])
    except:
            data.update({"Franked Amount" : "ERROR GO FIND IT YOURSELF"})
            data.update({"Unfranked Amount" : "ERROR GO FIND IT YOURSELF"})

    # Update with imputation credits
    if int(main_str.find("Part3E-Other-")) == int(-1):
        data["Imputation Credits"] = float(0)
    else:
        try:
            credits = str(main_str.split("ImputationCreditof")[1].split("perordinaryshare")[0])
            data["Imputation Credits"] = credits
        except:
            data["Imputation Credits"] = float(0)
            
    temp = pd.DataFrame.from_dict(data, orient='index', columns = [""]) 
    return temp

In [61]:
counter = 0
for file in glob.glob("*.pdf"):
    counter += 1
    print("Extracting File (", counter, "/", len(glob.glob("*.pdf")), ") - ", file)
    df = pd.concat([df, extract_info(file)],axis=1, sort=False)

df.to_csv('output.csv')
print("Saved to CSV!")

Extracting File ( 1 / 59 ) -  test 5.pdf
Extracting File ( 2 / 59 ) -  GTY 29Jun17.pdf
Extracting File ( 3 / 59 ) -  LNK 27Mar18.pdf
Extracting File ( 4 / 59 ) -  HPI 28Dec17.pdf
Extracting File ( 5 / 59 ) -  LLC 27Feb18.pdf
Extracting File ( 6 / 59 ) -  JIN 24Jul17.pdf
Extracting File ( 7 / 59 ) -  IPL 23Nov17.pdf
Extracting File ( 8 / 59 ) -  IDR 29Jun17.pdf
Extracting File ( 9 / 59 ) -  test 7.pdf
Extracting File ( 10 / 59 ) -  JIN 8Mar18.pdf
Extracting File ( 11 / 59 ) -  HLO 23Feb18.pdf
Extracting File ( 12 / 59 ) -  HLO 1Sep17.pdf
Extracting File ( 13 / 59 ) -  KMD 1Jun17.pdf
Extracting File ( 14 / 59 ) -  HPI 29Jun17.pdf
Extracting File ( 15 / 59 ) -  IGO 6Sep17.pdf
Extracting File ( 16 / 59 ) -  IDR 28Sep17.pdf
Extracting File ( 17 / 59 ) -  JIN 31Aug17.pdf
Extracting File ( 18 / 59 ) -  HOM 6Mar18.pdf
Extracting File ( 19 / 59 ) -  GMG 28Dec17.pdf
Extracting File ( 20 / 59 ) -  JHG 17Nov17.pdf
Extracting File ( 21 / 59 ) -  IRI 1Sep17.pdf
Extracting File ( 22 / 59 ) -  AOF.pdf

In [18]:
main_pdf = PyPDF2.PdfFileReader("aad.pdf", 'rb')
main_str = str()

# decrypt pdf with "" as password
if main_pdf.isEncrypted:
    main_pdf.decrypt("")

for pages in range(main_pdf.getNumPages()):
    p = main_pdf.getPage(pages)
    p_text= p.extractText()
    p_lines=str(p_text.splitlines())

    if pages == 0:
        main_str += p_lines
    else:
        main_str += " \n "
        main_str += p_lines

    # Create dictionary to convert to dataframe later
    data = {
        # Change names if ASX changes their format

        'Entity Name' : (main_str.split("Entity name")[-1].split("Security on which the Distribution")[0]),
        'Ticker' : (main_str.split("Security on which the Distribution will be paid")[-1])[:3],
        'Record Date' : (main_str.split("2A.4 +Record Date")[-1].split("2A.5 Ex Date")[0]),
        'Ex Date' : (main_str.split("2A.5 Ex Date")[-1].split("2A.6 Payment Date")[0]),
        'Payment Date' : (main_str.split("2A.6 Payment Date")[-1].split("2A.7 Are any of the")[0]),
        'Ex Date' : (main_str.split("2A.5 Ex Date")[-1].split("2A.6 Payment Date")[0])
        }

        
    main_str = main_str.replace(" ", "")
    main_str = main_str.replace("Notificationofdividend/distributionNotificationofdividend/distribution", "")
    
    # Add Primary Currency
    if ((main_str.split("Currencyinwhichthedividend/distributionismade")[-1][19:22])[0]) == ")":
        data["Primary Currency"] = (main_str.split("Currencyinwhichthedividend/distributionismade")[-1][20:23])
    else:
        data["Primary Currency"] = ((main_str.split("Currencyinwhichthedividend/distributionismade")[-1][19:22]))
        


In [44]:
main_str.split("Ordinarydividend/distributionfrankedamountper+security")[1][3:].split(".")[0] + "." + main_str.split("Ordinarydividend/distributionfrankedamountper+security")[1][3:].split(".")[1][:8]

'0.000000003'